# PyIceberg - Apache Iceberg com Python

Este notebook cobre:
- O que é Apache Iceberg
- Configuração do catálogo
- Criação e gerenciamento de tabelas
- Time Travel e Snapshots
- Schema Evolution
- Partitioning e Hidden Partitioning
- Integração com Spark

## O que é Apache Iceberg?

Apache Iceberg é um **formato de tabela aberto** para datasets analíticos grandes. 

### Vantagens:
- **ACID Transactions**: Operações atômicas
- **Time Travel**: Acesso a versões históricas
- **Schema Evolution**: Alterar schema sem reescrever dados
- **Hidden Partitioning**: Particionamento transparente
- **Otimizações**: Compaction, file pruning, etc.

### Comparação com Delta Lake:
| Feature | Iceberg | Delta Lake |
|---------|---------|------------|
| Open Source | Apache | Databricks |
| Engines | Multi-engine | Spark focused |
| Hidden Partitioning | Sim | Não |
| Partition Evolution | Sim | Limitado |

## 1. Instalação e Setup

```bash
pip install pyiceberg
pip install pyiceberg[s3,glue]  # Com extras para AWS
pip install pyiceberg[pyarrow]  # Com PyArrow
```

In [ ]:
# Imports básicos
from pyiceberg.catalog import load_catalog
from pyiceberg.schema import Schema
from pyiceberg.types import (
    NestedField, StringType, LongType, DoubleType, 
    TimestampType, DateType, BooleanType
)
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.transforms import YearTransform, MonthTransform, DayTransform, IdentityTransform

## 2. Configurando Catálogos

PyIceberg suporta vários tipos de catálogo:
- **SQL Catalog**: SQLite, PostgreSQL
- **Glue Catalog**: AWS Glue
- **Hive Catalog**: Hive Metastore
- **REST Catalog**: REST API

In [ ]:
# Catálogo SQLite (local, para desenvolvimento)
catalog_sqlite = load_catalog(
    "sqlite_catalog",
    **{
        "type": "sql",
        "uri": "sqlite:///./data/iceberg_catalog.db",
        "warehouse": "./data/iceberg_warehouse"
    }
)

In [ ]:
# Catálogo AWS Glue (produção)
# catalog_glue = load_catalog(
#     "glue_catalog",
#     **{
#         "type": "glue",
#         "warehouse": "s3://my-bucket/iceberg-warehouse",
#         "region_name": "us-east-1"
#     }
# )

In [ ]:
# Catálogo REST (Tabular, Nessie, etc.)
# catalog_rest = load_catalog(
#     "rest_catalog",
#     **{
#         "type": "rest",
#         "uri": "http://localhost:8181",
#         "warehouse": "s3://my-bucket/warehouse"
#     }
# )

## 3. Definindo Schema

In [ ]:
# Schema de exemplo - Vendas
schema = Schema(
    NestedField(field_id=1, name="id", field_type=LongType(), required=True),
    NestedField(field_id=2, name="produto", field_type=StringType(), required=True),
    NestedField(field_id=3, name="categoria", field_type=StringType(), required=False),
    NestedField(field_id=4, name="quantidade", field_type=LongType(), required=True),
    NestedField(field_id=5, name="preco_unitario", field_type=DoubleType(), required=True),
    NestedField(field_id=6, name="data_venda", field_type=DateType(), required=True),
    NestedField(field_id=7, name="regiao", field_type=StringType(), required=False),
)

print(schema)

## 4. Partitioning

### Hidden Partitioning
Iceberg suporta **hidden partitioning** - usuários não precisam saber como os dados são particionados.

### Transforms disponíveis:
- `identity`: valor exato
- `year`, `month`, `day`, `hour`: para timestamps/dates
- `bucket(n)`: hash em n buckets
- `truncate(n)`: primeiros n caracteres

In [ ]:
# Partition Spec com transforms
partition_spec = PartitionSpec(
    PartitionField(
        source_id=6,  # data_venda
        field_id=1000,
        transform=MonthTransform(),
        name="data_venda_month"
    ),
    PartitionField(
        source_id=7,  # regiao
        field_id=1001,
        transform=IdentityTransform(),
        name="regiao"
    )
)

print(partition_spec)

## 5. Criando e Gerenciando Tabelas

In [ ]:
# Criar namespace (database)
try:
    catalog_sqlite.create_namespace("vendas_db")
except Exception as e:
    print(f"Namespace já existe ou erro: {e}")

# Listar namespaces
print("Namespaces:", catalog_sqlite.list_namespaces())

In [ ]:
# Criar tabela
table_name = "vendas_db.vendas"

try:
    table = catalog_sqlite.create_table(
        identifier=table_name,
        schema=schema,
        partition_spec=partition_spec,
        properties={
            "write.format.default": "parquet",
            "write.parquet.compression-codec": "snappy"
        }
    )
    print(f"Tabela {table_name} criada!")
except Exception as e:
    print(f"Tabela já existe: {e}")
    table = catalog_sqlite.load_table(table_name)

In [ ]:
# Listar tabelas
print("Tabelas:", catalog_sqlite.list_tables("vendas_db"))

# Carregar tabela existente
table = catalog_sqlite.load_table(table_name)
print(f"\nSchema: {table.schema()}")
print(f"\nPartition Spec: {table.spec()}")

## 6. Inserindo Dados

In [ ]:
import pyarrow as pa
from datetime import date

# Criar dados com PyArrow
data = pa.table({
    "id": [1, 2, 3, 4, 5],
    "produto": ["Notebook", "Mouse", "Teclado", "Monitor", "Webcam"],
    "categoria": ["Computadores", "Periféricos", "Periféricos", "Computadores", "Periféricos"],
    "quantidade": [10, 50, 30, 15, 25],
    "preco_unitario": [3500.0, 150.0, 300.0, 1200.0, 400.0],
    "data_venda": [date(2024, 1, 15), date(2024, 1, 20), date(2024, 2, 10), date(2024, 2, 15), date(2024, 3, 1)],
    "regiao": ["Sul", "Sudeste", "Sul", "Norte", "Sudeste"]
})

# Append dados
table.append(data)
print("Dados inseridos!")

In [ ]:
# Overwrite (substitui todos os dados)
# table.overwrite(data)

# Delete (baseado em expressão)
# from pyiceberg.expressions import EqualTo
# table.delete(EqualTo("regiao", "Norte"))

## 7. Lendo Dados

In [ ]:
# Scan completo
scan = table.scan()
df = scan.to_arrow()
print(df.to_pandas())

In [ ]:
# Scan com seleção de colunas
scan = table.scan(selected_fields=("produto", "quantidade", "preco_unitario"))
df = scan.to_arrow()
print(df.to_pandas())

In [ ]:
from pyiceberg.expressions import GreaterThan, EqualTo, And

# Scan com filtros (row filter)
scan = table.scan(
    row_filter=And(
        GreaterThan("quantidade", 20),
        EqualTo("regiao", "Sul")
    )
)
df = scan.to_arrow()
print(df.to_pandas())

## 8. Time Travel e Snapshots

In [ ]:
# Listar snapshots
for snapshot in table.snapshots():
    print(f"Snapshot ID: {snapshot.snapshot_id}")
    print(f"  Timestamp: {snapshot.timestamp_ms}")
    print(f"  Operation: {snapshot.summary}")
    print()

In [ ]:
# Inserir mais dados para criar novo snapshot
novos_dados = pa.table({
    "id": [6, 7],
    "produto": ["Headset", "SSD"],
    "categoria": ["Periféricos", "Armazenamento"],
    "quantidade": [40, 20],
    "preco_unitario": [250.0, 500.0],
    "data_venda": [date(2024, 3, 15), date(2024, 3, 20)],
    "regiao": ["Sul", "Sudeste"]
})

table.append(novos_dados)
print("Novos dados inseridos!")

In [ ]:
# Time travel - ler snapshot específico
snapshots = list(table.snapshots())
if len(snapshots) > 1:
    old_snapshot_id = snapshots[0].snapshot_id
    
    # Scan de snapshot antigo
    scan = table.scan(snapshot_id=old_snapshot_id)
    df_old = scan.to_arrow()
    print(f"Dados do snapshot {old_snapshot_id}:")
    print(df_old.to_pandas())

In [ ]:
# Dados atuais
scan = table.scan()
df_current = scan.to_arrow()
print("Dados atuais:")
print(df_current.to_pandas())

## 9. Schema Evolution

In [ ]:
# Adicionar coluna
with table.update_schema() as update:
    update.add_column("desconto", DoubleType(), doc="Desconto aplicado")

print("Nova coluna adicionada!")
print(table.schema())

In [ ]:
# Renomear coluna
# with table.update_schema() as update:
#     update.rename_column("preco_unitario", "valor_unitario")

# Alterar tipo (upcasting permitido)
# with table.update_schema() as update:
#     update.update_column("quantidade", DoubleType())

# Tornar nullable
# with table.update_schema() as update:
#     update.make_column_optional("quantidade")

## 10. Partition Evolution

In [ ]:
# Ver particionamento atual
print("Particionamento atual:")
print(table.spec())

In [ ]:
# Evoluir particionamento (novos dados usarão novo spec)
# with table.update_spec() as update:
#     update.add_field(
#         source_column_name="categoria",
#         transform=IdentityTransform(),
#         partition_field_name="categoria"
#     )

## 11. Integração com Spark

In [ ]:
# Configuração Spark com Iceberg
# from pyspark.sql import SparkSession

# spark = SparkSession.builder \
#     .appName("IcebergSpark") \
#     .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.0") \
#     .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog") \
#     .config("spark.sql.catalog.spark_catalog.type", "hive") \
#     .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
#     .config("spark.sql.catalog.local.type", "hadoop") \
#     .config("spark.sql.catalog.local.warehouse", "./iceberg-warehouse") \
#     .getOrCreate()

In [ ]:
# Operações SQL com Iceberg no Spark

# Criar tabela
# spark.sql("""
#     CREATE TABLE local.db.vendas (
#         id BIGINT,
#         produto STRING,
#         valor DOUBLE,
#         data_venda DATE
#     )
#     USING iceberg
#     PARTITIONED BY (months(data_venda))
# """)

# Inserir dados
# spark.sql("""
#     INSERT INTO local.db.vendas VALUES
#     (1, 'Notebook', 3500.0, DATE '2024-01-15'),
#     (2, 'Mouse', 150.0, DATE '2024-02-20')
# """)

# Time travel
# spark.sql("SELECT * FROM local.db.vendas VERSION AS OF 1")
# spark.sql("SELECT * FROM local.db.vendas TIMESTAMP AS OF '2024-01-01 00:00:00'")

# Histórico
# spark.sql("SELECT * FROM local.db.vendas.history")
# spark.sql("SELECT * FROM local.db.vendas.snapshots")
# spark.sql("SELECT * FROM local.db.vendas.files")

## 12. Manutenção de Tabelas

In [ ]:
# Metadados da tabela
print(f"Location: {table.location()}")
print(f"Current Snapshot: {table.current_snapshot()}")
print(f"Properties: {table.properties}")

In [ ]:
# Via Spark SQL - Manutenção

# Expire snapshots antigos
# spark.sql("CALL local.system.expire_snapshots('db.vendas', TIMESTAMP '2024-01-01')")

# Remover arquivos órfãos
# spark.sql("CALL local.system.remove_orphan_files('db.vendas')")

# Reescrever arquivos (compaction)
# spark.sql("CALL local.system.rewrite_data_files('db.vendas')")

# Reescrever manifests
# spark.sql("CALL local.system.rewrite_manifests('db.vendas')")

## Resumo de Comandos PyIceberg

```python
# Catálogo
catalog = load_catalog("name", **config)
catalog.list_namespaces()
catalog.create_namespace("db")
catalog.list_tables("db")

# Tabela
table = catalog.create_table(identifier, schema, partition_spec)
table = catalog.load_table("db.tabela")

# CRUD
table.append(pyarrow_table)
table.overwrite(pyarrow_table)
table.delete(expression)

# Scan
scan = table.scan(selected_fields, row_filter, snapshot_id)
df = scan.to_arrow()

# Schema Evolution
with table.update_schema() as update:
    update.add_column("col", Type())
    update.rename_column("old", "new")

# Snapshots
table.snapshots()
table.current_snapshot()
```